In [21]:
import pandas as pd
import numpy as np
from itertools import product
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from scipy.signal import savgol_filter
import pickle
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

# DataSets

In [9]:
orbital_file = r"C:\Users\Suare\satellite_anomaly_project\data\cleaned\all_satellite_orbitals.csv"
maneuver_file = r"C:\Users\Suare\satellite_anomaly_project\data\cleaned\cleaned_maneuvers.csv"
df_orbital = pd.read_csv(orbital_file)
df_maneuver = pd.read_csv(maneuver_file)

In [11]:
#Filter the dataset with the satellites selected for the report
df_orbital_filtered = df_orbital[
    df_orbital['satellite_name'].isin(["Fengyun_2E", "Fengyun_2F","Fengyun_4A", "Jason_3", "Sentinel_3A","Sentinel_3B","CryoSat_2"])]

In [39]:
# Ensure 'start_date' is datetime
df_maneuver['start_date'] = pd.to_datetime(df_maneuver['start_date'], errors='coerce')

# Build filtered ground truth dictionary
ground_truth_dict = {}

for r in arima_results_final3:
    satellite = r['satellite']
    residuals_index = r['residuals'].index
    start_date = residuals_index.min()
    end_date = residuals_index.max()

    satellite_maneuvers = df_maneuver[df_maneuver['OrbitalKeyName'] == satellite]
    # Convert to datetime
    satellite_maneuvers['start_date'] = pd.to_datetime(satellite_maneuvers['start_date'], errors='coerce')

    valid_maneuvers = satellite_maneuvers[
        (satellite_maneuvers['start_date'] >= start_date) &
        (satellite_maneuvers['start_date'] <= end_date)
    ]['start_date'].dropna().sort_values()

    ground_truth_dict[satellite] = valid_maneuvers.tolist()

In [15]:
def apply_mad_special(series_mm, satellite, variable, sigma_threshold=10):
    """
    Note: Contains special handling for CryoSat_2 Brouwer mean motion due to 
    regime change on 2020-07-17. This should be generalized in future versions.
    """
    series_with_nan = series_mm.copy()
    
    # SPECIAL CASE: CryoSat_2 Brouwer mean motion (regime change handling)
    if satellite == "CryoSat_2" and variable == "Brouwer mean motion":
        breakpoint_date = pd.Timestamp('2020-07-17')
        
        regime1_mask = series_mm.index < breakpoint_date
        regime2_mask = series_mm.index >= breakpoint_date
        
        all_outliers = []
        
        for i, regime_mask in enumerate([regime1_mask, regime2_mask], 1):
            regime_data = series_mm[regime_mask]
            
            if len(regime_data) == 0:
                continue
            
            median_val = regime_data.median()
            sigma = (regime_data - median_val).abs().median() * 1.4826
            if pd.isna(sigma) or sigma == 0:
                sigma = regime_data.std()
            
            outlier_mask_regime = (regime_data - median_val).abs() > sigma_threshold * sigma
            regime_outliers = regime_data[outlier_mask_regime]
            
            # Replace outliers with NaN in the full series
            series_with_nan.loc[regime_mask & (series_mm.index.isin(regime_outliers.index))] = np.nan
            all_outliers.append(regime_outliers)
        
        outliers = pd.concat(all_outliers).sort_index() if any(len(o) > 0 for o in all_outliers) else pd.Series(dtype=float)
    
    return series_with_nan, outliers

In [27]:
def run_arima_experiments_dual_mad(df_orbital_filtered):
    results = []
    satellites = df_orbital_filtered['satellite_name'].unique()
    variables = ["Brouwer mean motion", "eccentricity", "argument of perigee", "right ascension", "inclination", "mean anomaly"]
    smooth_windows = [7, 9]
    sigma_thresholds = [10, 20]

    for satellite, variable in product(satellites, variables):
        print(f"\n--- Processing {satellite} - {variable} ---")
        
        df = df_orbital_filtered[df_orbital_filtered['satellite_name'] == satellite][['epoch', variable]].copy()
        df['epoch'] = pd.to_datetime(df['epoch'])
        df.set_index('epoch', inplace=True)
        df_mm = (df - df.mean()) * 1e7
        df_resampled = df_mm.resample('D').mean()
        df_filled = df_resampled.interpolate(method='bfill')

        adf_result = adfuller(df_filled[variable].dropna())
        p_value = adf_result[1]
        stationary = p_value < 0.05
        df_base = df_filled.copy() if stationary else df_filled.diff().dropna()

        # Precompute MAD scenarios
        mad_scenarios = {}
        for sigma in sigma_thresholds:
            if satellite == "CryoSat_2" and variable == "Brouwer mean motion":
                series_with_nan, outliers = apply_mad_special(df_mm[variable], satellite, variable, sigma_threshold=sigma)
            else:
                median_val = df_mm[variable].median()
                mad = (df_mm[variable] - median_val).abs().median() * 1.4826
                if pd.isna(mad) or mad == 0:
                    mad = df_mm[variable].std()
                outlier_mask = (df_mm[variable] - median_val).abs() > sigma * mad
                outliers = df_mm[variable][outlier_mask]
                series_with_nan = df_mm[variable].copy()
                series_with_nan.loc[outlier_mask] = np.nan
            
            df_mad = series_with_nan.resample('D').mean().interpolate(method='bfill').to_frame(name=variable)
            mad_scenarios[sigma] = (df_mad, outliers)

        # Scenario 4: Log transform
        try:
            df_log_raw = np.log(df)
            df_mm_log = (df_log_raw - df_log_raw.mean()) * 1e7
            df_log = df_mm_log.resample('D').mean().interpolate(method='bfill')
        except:
            df_log = None

        scenario_data = {
            1: (df_base.copy(), 0),
            2: (df_base.copy(), 1)
        }
        
        # Store outliers separately to avoid overwriting
        scenario_outliers = {}

        # Add MAD variants
        for sigma, (df_mad, outliers) in mad_scenarios.items():
            scen_label = f"Scenario 3 (MAD σ={sigma})"
            scenario_data[scen_label] = (df_mad.copy(), 0 if stationary else 1)
            scenario_outliers[scen_label] = outliers

        # Add log scenario
        if df_log is not None:
            scenario_data[4] = (df_log.copy(), 0 if stationary else 1)

        for scen, (ts, d) in scenario_data.items():
            print(f"  Running {scen}...")
            try:
                model = auto_arima(ts, d=d, start_p=0, max_p=5, start_q=0, max_q=5,
                                   seasonal=False, information_criterion='aic', stepwise=False,
                                   error_action='ignore', suppress_warnings=True)

                lb_test = acorr_ljungbox(model.resid(), lags=[10], return_df=True)
                lb_pvalue = lb_test['lb_pvalue'].iloc[0]

                results.append({
                    "satellite": satellite,
                    "variable": variable,
                    "scenario":  scen if isinstance(scen, str) else f"Scenario {scen}",
                    "ARIMA_order": model.order,
                    "aic": model.aic(),
                    "ljung_box_pvalue": lb_pvalue,
                    "residuals": model.resid(),
                    "fitted_values": model.fittedvalues,
                    "series_original": None,
                    "input_series": ts,
                    "series_outliers": scenario_outliers.get(scen, None)
                })
            except Exception as e:
                print(f"ARIMA error: {satellite}-{variable}-{scen}: {e}")

        # Scenario 5: smoothing for stationary Brouwer only
        if stationary and variable == "Brouwer mean motion":
            for win in smooth_windows:
                print(f"  Running Scenario 5 (win={win})...")
                try:
                    smooth_series = savgol_filter(df_filled[variable].values, window_length=win, polyorder=1)
                    ts_smooth = pd.Series(smooth_series, index=df_filled.index)
                    model = auto_arima(ts_smooth, d=0, start_p=0, max_p=5, start_q=0, max_q=5,
                                       seasonal=False, information_criterion='aic', stepwise=False,
                                       error_action='ignore', suppress_warnings=True)

                    lb_test = acorr_ljungbox(model.resid(), lags=[10], return_df=True)
                    lb_pvalue = lb_test['lb_pvalue'].iloc[0]

                    results.append({
                        "satellite": satellite,
                        "variable": variable,
                        "scenario": f"Scenario 5 (win={win})",
                        "ARIMA_order": model.order,
                        "aic": model.aic(),
                        "ljung_box_pvalue": lb_pvalue,
                        "residuals": model.resid(),
                        "fitted_values": model.fittedvalues,
                        "series_original": df_filled[variable],
                        "input_series": ts_smooth,
                        "series_outliers": None
                    })
                except Exception as e:
                    print(f"ARIMA error: {satellite}-{variable}-Scenario 5 win={win}: {e}")
        else:
            print("  Skipping Scenario 5")

    print(f"\n Completed! Total results: {len(results)}")
    return results

In [29]:
arima_results_final3 = run_arima_experiments_dual_mad(df_orbital_filtered)


--- Processing CryoSat_2 - Brouwer mean motion ---
  Running 1...
  Running 2...
  Running Scenario 3 (MAD σ=10)...
  Running Scenario 3 (MAD σ=20)...
  Running 4...
  Skipping Scenario 5

--- Processing CryoSat_2 - eccentricity ---
  Running 1...
  Running 2...
  Running Scenario 3 (MAD σ=10)...
  Running Scenario 3 (MAD σ=20)...
  Running 4...
  Skipping Scenario 5

--- Processing CryoSat_2 - argument of perigee ---
  Running 1...
  Running 2...
  Running Scenario 3 (MAD σ=10)...
  Running Scenario 3 (MAD σ=20)...
  Running 4...
  Skipping Scenario 5

--- Processing CryoSat_2 - right ascension ---
  Running 1...
  Running 2...
  Running Scenario 3 (MAD σ=10)...
  Running Scenario 3 (MAD σ=20)...
  Running 4...
  Skipping Scenario 5

--- Processing CryoSat_2 - inclination ---
  Running 1...
  Running 2...
  Running Scenario 3 (MAD σ=10)...
  Running Scenario 3 (MAD σ=20)...
  Running 4...
  Skipping Scenario 5

--- Processing CryoSat_2 - mean anomaly ---
  Running 1...
  Running 2...

In [31]:
# Save the summary DataFrame
with open('arima_model_results3.pkl', 'wb') as f:
    pickle.dump(arima_results_final3, f)

In [23]:
summary = pd.DataFrame([{
    'satellite': r['satellite'],
    'variable': r['variable'],
    'scenario': r['scenario'],
    'ARIMA_order': r['ARIMA_order'],
    'aic': r['aic'],
    'ljung_box_pvalue': r['ljung_box_pvalue'],
    'n_residuals': len(r['residuals']),
    'start_epoch': r['residuals'].index.min(),
    'end_epoch': r['residuals'].index.max(),
    'outliers': None if r['series_outliers'] is None else len(r['series_outliers']),
    'min_resid': r['residuals'].min(),
    'max_resid': r['residuals'].max()
} for r in arima_results_final3])

In [25]:
summary

,satellite,variable,scenario,ARIMA_order,aic,ljung_box_pvalue,n_residuals,start_epoch,end_epoch,outliers,min_resid,max_resid
0,CryoSat_2,Brouwer mean motion,1,"(2, 0, 2)",21560.370871,4.664282e-58,4539,2010-04-26,2022-09-28,NaN,-7.383410e+01,8.789055e+01
1,CryoSat_2,Brouwer mean motion,2,"(5, 1, 0)",22219.958838,5.185598e-81,4539,2010-04-26,2022-09-28,NaN,-7.102850e+01,9.556193e+01
2,CryoSat_2,Brouwer mean motion,Scenario 3 (MAD σ=10),"(0, 1, 0)",17819.531174,7.441187e-01,4529,2010-05-06,2022-09-28,39.0,-1.041693e+02,3.270596e+01
3,CryoSat_2,Brouwer mean motion,Scenario 3 (MAD σ=20),"(1, 1, 0)",18444.209879,9.995865e-01,4540,2010-04-25,2022-09-28,39.0,-1.041734e+02,2.788001e+01
4,CryoSat_2,Brouwer mean motion,4,"(2, 1, 2)",46601.609004,5.018910e-58,4540,2010-04-25,2022-09-28,NaN,-1.164112e+03,1.386650e+03
...,...,...,...,...,...,...,...,...,...,...,...,...
217,Sentinel_3B,mean anomaly,1,"(2, 0, 3)",49111.238564,8.127252e-10,1604,2018-05-10,2022-09-29,NaN,-2.350455e+07,1.843505e+07
218,Sentinel_3B,mean anomaly,2,"(1, 1, 4)",49143.235074,7.302101e-08,1604,2018-05-10,2022-09-29,NaN,-2.357785e+07,1.942273e+07
219,Sentinel_3B,mean anomaly,Scenario 3 (MAD σ=10),"(2, 0, 3)",49111.238564,8.127252e-10,1604,2018-05-10,2022-09-29,0.0,-2.350455e+07,1.843505e+07
220,Sentinel_3B,mean anomaly,Scenario 3 (MAD σ=20),"(2, 0, 3)",49111.238564,8.127252e-10,1604,2018-05-10,2022-09-29,0.0,-2.350455e+07,1.843505e+07


# Quantitive Evaluation

In [47]:
def convert_timestamp_series_to_epoch(series):
    return (
        (series - pd.Timestamp(year=1970, month=1, day=1)) // pd.Timedelta(seconds=1)
    ).values
    
def compute_simple_matching_precision_recall_for_one_threshold(
    matching_max_days,
    threshold,
    series_ground_truth_manoeuvre_timestamps,
    series_predictions,
):
    """
    :param matching_max_days
    :param threshold
    :param series_ground_truth_manoeuvre_timestamps
    :param series_predictions: The index of this series should be the timestamps of the predictions.
    :return: (precision, recall)

   Computes the precision and recall at one anomaly threshold.

   Does this using an implementation of the framework proposed by Zhao:
   Zhao, L. (2021). Event prediction in the big data era: A systematic survey. ACM Computing Surveys (CSUR), 54(5), 1-37.
   https://doi.org/10.1145/3450287

   The method matches each manoeuvre prediction with the closest ground-truth manoeuvre, if it is within a time window.

   Predictions with a match are then true positives and those without a match are false positives. Ground-truth manoeuvres
   with no matching prediction are counted as false negatives.
   """

    matching_max_distance_seconds = pd.Timedelta(days=matching_max_days).total_seconds()

    dict_predictions_to_ground_truth = {}
    dict_ground_truth_to_predictions = {}

    manoeuvre_timestamps_seconds = convert_timestamp_series_to_epoch(series_ground_truth_manoeuvre_timestamps)
    pred_time_stamps_seconds = convert_timestamp_series_to_epoch(series_predictions.index)
    predictions = series_predictions.to_numpy()

    for i in range(predictions.shape[0]):
        if predictions[i] >= threshold:
            left_index = np.searchsorted(
                manoeuvre_timestamps_seconds, pred_time_stamps_seconds[i]
            )

            if left_index != 0:
                left_index -= 1

            index_of_closest = left_index

            if (left_index < series_ground_truth_manoeuvre_timestamps.shape[0] - 1) and (
                abs(manoeuvre_timestamps_seconds[left_index] - pred_time_stamps_seconds[i])
                > abs(manoeuvre_timestamps_seconds[left_index + 1] - pred_time_stamps_seconds[i])
            ):
                index_of_closest = left_index + 1

            diff = abs(manoeuvre_timestamps_seconds[index_of_closest] - pred_time_stamps_seconds[i])

            if diff < matching_max_distance_seconds:
                dict_predictions_to_ground_truth[i] = (
                    index_of_closest,
                    diff,
                )
                if index_of_closest in dict_ground_truth_to_predictions:
                    dict_ground_truth_to_predictions[index_of_closest].append(i)
                else:
                    dict_ground_truth_to_predictions[index_of_closest] = [i]

    positive_prediction_indices = np.argwhere(predictions >= threshold)[:, 0]
    list_false_positives = [
        pred_ind for pred_ind in positive_prediction_indices if pred_ind not in dict_predictions_to_ground_truth.keys()
    ]
    list_false_negatives = [
        true_ind for true_ind in np.arange(0, len(series_ground_truth_manoeuvre_timestamps))
        if true_ind not in dict_ground_truth_to_predictions.keys()
    ]

    tp = len(dict_ground_truth_to_predictions)
    fp = len(list_false_positives)
    fn = len(list_false_negatives)


    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall, tp, fp, fn

In [49]:
def evaluate_all_thresholds_with_flags_arima(
    results,
    ground_truth_dict,
    min_precision_best_recall=0.60
):
    summary = []
    pr_data = []

    for r in results:
        variable = r['variable']
        satellite = r['satellite']
        scenario = r['scenario']
        abs_resid = r['residuals'].abs()
        timestamps = abs_resid.index
        series_predictions = pd.Series(abs_resid.values, index=timestamps)

        if satellite not in ground_truth_dict:
            continue

        filtered_ground_truth = pd.Series(ground_truth_dict[satellite])


        # Define threshold range
        thresholds = series_predictions[
            (series_predictions > series_predictions.quantile(0.5)) &
            (series_predictions < series_predictions.quantile(0.99))
        ].unique()
        thresholds = sorted(thresholds, reverse=True)

        # Define matching days based on variable
        matching_days_list = [1, 2, 3, 4] if variable == "Brouwer mean motion" else [3]

        for match_days in matching_days_list:
            print(f"Processing: {satellite} - {variable} - {scenario} - MatchDays={match_days}")

            precision_list = []
            recall_list = []
            threshold_list = []
            tp_list = []
            fp_list = []
            fn_list = []

            best_f1 = 0
            best_f2 = 0
            best_recall_over_min_precision = 0

            best_row_f1 = {}
            best_row_f2 = {}
            best_row_recall_strategy = {}

            for threshold in thresholds:
                precision, recall, tp, fp, fn = compute_simple_matching_precision_recall_for_one_threshold(
                    matching_max_days=match_days,
                    threshold=threshold,
                    series_ground_truth_manoeuvre_timestamps=filtered_ground_truth,
                    series_predictions=series_predictions
                )

                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                f2 = (5 * precision * recall) / ((4 * precision) + recall) if (precision + recall) > 0 else 0

                precision_list.append(precision)
                recall_list.append(recall)
                threshold_list.append(threshold)
                tp_list.append(tp)
                fp_list.append(fp)
                fn_list.append(fn)

                if f1 > best_f1:
                    best_f1 = f1
                    best_row_f1 = {
                        "threshold_f1": threshold,
                        "recall_f1": recall,
                        "precision_f1": precision,
                        "f1_score": f1,
                        "tp_f1": tp,
                        "fp_f1": fp,
                        "fn_f1": fn
                    }

                if f2 > best_f2:
                    best_f2 = f2
                    best_row_f2 = {
                        "threshold_f2": threshold,
                        "recall_f2": recall,
                        "precision_f2": precision,
                        "f2_score": f2,
                        "tp_f2": tp,
                        "fp_f2": fp,
                        "fn_f2": fn
                    }

                if precision >= min_precision_best_recall and recall > best_recall_over_min_precision:
                    best_recall_over_min_precision = recall
                    best_row_recall_strategy = {
                        "threshold_best_recall_strategy": threshold,
                        "recall_best_recall_strategy": recall,
                        "precision_best_recall_strategy": precision,
                        "tp_best_recall_strategy": tp,
                        "fp_best_recall_strategy": fp,
                        "fn_best_recall_strategy": fn
                    }

            if best_row_f1 and best_row_f2:
                combined_row = {
                    "satellite": satellite,
                    "variable": variable,
                    "scenario": scenario,
                    "matching_max_days": match_days,
                    **best_row_f1,
                    **best_row_f2,
                    **(best_row_recall_strategy if best_row_recall_strategy else {
                        "threshold_best_recall_strategy": None,
                        "recall_best_recall_strategy": None,
                        "precision_best_recall_strategy": None,
                        "tp_best_recall_strategy": None,
                        "fp_best_recall_strategy": None,
                        "fn_best_recall_strategy": None
                    })
                }
                summary.append(combined_row)

            pr_data.append({
                "satellite": satellite,
                "variable": variable,
                "scenario": scenario,
                "matching_max_days": match_days,
                "thresholds": threshold_list,
                "precision": precision_list,
                "recall": recall_list,
                "tp": tp_list,
                "fp": fp_list,
                "fn": fn_list
            })

    df_summary = pd.DataFrame(summary)
    return df_summary, pr_data

In [51]:
df_arima_summary3, pr_data_arima3 = evaluate_all_thresholds_with_flags_arima(
    results=arima_results_final3,
    ground_truth_dict=ground_truth_dict
)

Processing: CryoSat_2 - Brouwer mean motion - Scenario 1 - MatchDays=1
Processing: CryoSat_2 - Brouwer mean motion - Scenario 1 - MatchDays=2
Processing: CryoSat_2 - Brouwer mean motion - Scenario 1 - MatchDays=3
Processing: CryoSat_2 - Brouwer mean motion - Scenario 1 - MatchDays=4
Processing: CryoSat_2 - Brouwer mean motion - Scenario 2 - MatchDays=1
Processing: CryoSat_2 - Brouwer mean motion - Scenario 2 - MatchDays=2
Processing: CryoSat_2 - Brouwer mean motion - Scenario 2 - MatchDays=3
Processing: CryoSat_2 - Brouwer mean motion - Scenario 2 - MatchDays=4
Processing: CryoSat_2 - Brouwer mean motion - Scenario 3 (MAD σ=10) - MatchDays=1
Processing: CryoSat_2 - Brouwer mean motion - Scenario 3 (MAD σ=10) - MatchDays=2
Processing: CryoSat_2 - Brouwer mean motion - Scenario 3 (MAD σ=10) - MatchDays=3
Processing: CryoSat_2 - Brouwer mean motion - Scenario 3 (MAD σ=10) - MatchDays=4
Processing: CryoSat_2 - Brouwer mean motion - Scenario 3 (MAD σ=20) - MatchDays=1
Processing: CryoSat_2 

In [53]:
# Save the summary DataFrame
with open('arima_pr_summary3.pkl', 'wb') as f:
    pickle.dump(df_arima_summary3, f)

In [123]:
with open('arima_pr_data3.pkl', 'wb') as f:
    pickle.dump(pr_data_arima3, f)

In [125]:
df_arima_summary3.to_csv('arima_metrics_pr3.csv', index=False)